#  Shex Creation Help

This tool will help to generate SheX for a given entity (Example E130.ipynb written by Tom Baker)

We would like to know the properties to be used for a given item. There are two ways: making use of Wikidata property suggestion API or checking the current usage of properties by sibling instances. 

* Obtaining suggestions for a given item

Given an item identifier (Q-number) on Wikidata,we would like to know the properties suggested by Wikidata. Wikidata has a suggestions operations for this purpose. 

But sometimes, it's useful to make use of existing instances of a class and the properties used by these instances. For example, give me the properties used by different instances of painting on Wikidata.

* Get properties of instances of a class

Given an item identifier (Q-number) on Wikidata, we would like to know the properties used by instances of this item

### Acknowledgements
Andra Waagmeester, Maarten Daammers

In [1]:
import requests

baseurl="https://www.wikidata.org/w/api.php?action=wbsgetsuggestions&format=json&limit=50&entity="

def get_property_suggestions(entity):
    """
    Returns string of property suggestion

    Parameters:
        entity (str): identifier of an identity

    Returns:
        (list): list of properties

    """
    
    propertylist = []
    url = baseurl + entity
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for prop in results["search"]:
            propertylist.append("wdt:" + prop["id"])
            
    return(propertylist)

In [21]:
l = get_property_suggestions("Q23898")
print(l)

['wdt:P1476']


In [17]:
from urllib.parse import quote
def get_currently_used_properties(entityid):
    propertylist = []
    query = """
       SELECT DISTINCT ?property
       {
            hint:Query hint:optimizer "None".
            {
                SELECT ?entity {
               ?entity wdt:P31 wd:entityid.
              }
              LIMIT 1000
            }

            ?entity ?prop [].
                ?property wikibase:directClaim ?prop.
       }"""
    baseurl = "https://query.wikidata.org/sparql?query="
    formatstr = "&format=json"

    query = query.replace("entityid", entityid)
    url = baseurl+quote(query)+formatstr
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for prop in results["results"]["bindings"]:
            propertylist.append(prop["property"]["value"].replace("http://www.wikidata.org/entity/","wdt:"))
    
    return(propertylist)

In [19]:
def get_properties(entity):
    propertylist = []
    try:
        propertylist.extend(get_property_suggestions(entity))
        propertylist.extend(get_currently_used_properties(entity))
    except Exception as e:
        print(e)
        pass
    return list(set(propertylist))

print(get_properties("Q3305213"))

['wdt:P6372', 'wdt:P4511', 'wdt:P1705', 'wdt:P186', 'wdt:P138', 'wdt:P4178', 'wdt:P4582', 'wdt:P6020', 'wdt:P508', 'wdt:P840', 'wdt:P1669', 'wdt:P1566', 'wdt:P2284', 'wdt:P3219', 'wdt:P1269', 'wdt:P6358', 'wdt:P2441', 'wdt:P88', 'wdt:P646', 'wdt:P528', 'wdt:P2092', 'wdt:P2344', 'wdt:P6405', 'wdt:P6108', 'wdt:P3931', 'wdt:P4380', 'wdt:P675', 'wdt:P109', 'wdt:P1028', 'wdt:P6002', 'wdt:P3893', 'wdt:P859', 'wdt:P1449', 'wdt:P18', 'wdt:P2610', 'wdt:P1071', 'wdt:P135', 'wdt:P4373', 'wdt:P6625', 'wdt:P6375', 'wdt:P3855', 'wdt:P214', 'wdt:P969', 'wdt:P941', 'wdt:P2511', 'wdt:P170', 'wdt:P2386', 'wdt:P551', 'wdt:P1889', 'wdt:P6573', 'wdt:P3504', 'wdt:P1207', 'wdt:P1259', 'wdt:P213', 'wdt:P888', 'wdt:P625', 'wdt:P2014', 'wdt:P131', 'wdt:P1545', 'wdt:P4905', 'wdt:P6058', 'wdt:P973', 'wdt:P4659', 'wdt:P1726', 'wdt:P1679', 'wdt:P6246', 'wdt:P462', 'wdt:P1684', 'wdt:P949', 'wdt:P1296', 'wdt:P156', 'wdt:P989', 'wdt:P189', 'wdt:P1476', 'wdt:P571', 'wdt:P6489', 'wdt:P5008', 'wdt:P5891', 'wdt:P641', 'wd

In [22]:
def get_property_datatype(entityid):
    csvresult =[]
    propertylist = get_properties(entityid)
    propertyliststr = ' '.join(propertylist) 
    query = """
    SELECT DISTINCT ("entityid" as ?entity) ?property ?datatype
    {
       VALUES ?prop {propertylist}.
       ?property wikibase:directClaim ?prop;
            wikibase:propertyType ?datatype.
    }"""
    
    baseurl = "https://query.wikidata.org/sparql?query="
    formatstr = "&format=json"

    query = query.replace("entityid", entityid)
    query = query.replace("propertylist", propertyliststr)
    url = baseurl+quote(query)+formatstr
    response = requests.get(url)
    if(response.status_code == 200):
        results = response.json()
        for result in results["results"]["bindings"]:
            csvresult.append(result["entity"]["value"]+ ", " +
                   result["property"]["value"].replace("http://www.wikidata.org/entity/", "wdt:")+ ", " +
                   result["datatype"]["value"])
    return(csvresult)
        
print('\n'.join(get_property_datatype("Q3305213")))

Q3305213, wdt:P6606, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P7420, http://wikiba.se/ontology#CommonsMedia
Q3305213, wdt:P7457, http://wikiba.se/ontology#CommonsMedia
Q3305213, wdt:P6595, http://wikiba.se/ontology#ExternalId
Q3305213, wdt:P6610, http://wikiba.se/ontology#ExternalId
Q3305213, wdt:P6625, http://wikiba.se/ontology#ExternalId
Q3305213, wdt:P6663, http://wikiba.se/ontology#ExternalId
Q3305213, wdt:P7229, http://wikiba.se/ontology#ExternalId
Q3305213, wdt:P275, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P276, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P361, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P366, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P460, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P462, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P495, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P527, http://wikiba.se/ontology#WikibaseItem
Q3305213, wdt:P551, http://wikiba.se/ontology#WikibaseItem